In [ ]:
import pandas as pd
import requests
import calendar

### DATOS HORARIOS POR PROVINCIA

# Con esta funcion sacamos un df con los datos de horas de luz de la provincia especificada.

def sacar_datos_horarios(ciudad):    
    # URL del archivo de texto
    url = f'https://cdn.mitma.gob.es/portal-web-drupal/salidapuestasol/2024/{ciudad}-2024.txt'

    # Descargar contenido
    response = requests.get(url)
    response.encoding = 'ISO-8859-1'
    content = response.text

    # Dividir en líneas y omitir las filas de encabezado y pie
    lines = content.splitlines()[4:6] + content.splitlines()[7:]  # Quitar encabezado

    # Crear listas de listas para estructurar datos manualmente,
    data = []

    for line in lines:
        # Trata de dividir las líneas de manera personalizada
        parts = line.split() 
        data.append(parts)

    # Ajustamos para los meses que no tienen ni 30 ni 31 dias

    def adjust_days(data):
    # Recorrer cada sublista en la lista de datos
        for sublist in data:
            # Verificar si la sublista corresponde al día 30
            if not sublist:
                continue
            elif sublist[0] == '30':
            # Corregimos Febrero (posiciones 3 y 4)
                sublist.insert(3, '0000')
                sublist.insert(4, '0000')
            elif sublist[0] == '31':
            # Corregimos Febrero, Abril, Junio, Septiembre y Noviembre
                sublist.insert(3, '0000')
                sublist.insert(4, '0000')
                sublist.insert(7, '0000')
                sublist.insert(8, '0000')
                sublist.insert(11, '0000')
                sublist.insert(12, '0000')
                sublist.insert(17, '0000')
                sublist.insert(18, '0000')
                sublist.insert(21, '0000')
                sublist.insert(22, '0000')
            
        return data
    
    data = adjust_days(data)

    # Columnas de cabeceras
    columnas = ['Dia', 'Ene_Ort', 'Ene_Ocas', 'Feb_Ort', 'Feb_Ocas', 'Mar_Ort', 'Mar_Ocas', 'Apr_Ort', 'Apr_Ocas', 
                'May_Ort', 'May_Ocas', 'Jun_Ort', 'Jun_Ocas', 'Jul_Ort', 'Jul_Ocas', 'Aug_Ort', 'Aug_Ocas',
            'Sep_Ort', 'Sep_Ocas', 'Oct_Ort', 'Oct_Ocas', 'Nov_Ort', 'Nov_Ocas', 'Dec_Ort', 'Dec_Ocas']

    # Crear DataFrame
    df = pd.DataFrame(data).iloc[:-4].dropna(axis=1, how='all')
    df.columns = columnas
    df = df.iloc[2:].reset_index(drop=True)

    def format_time(value):
        if isinstance(value, str):
            # Asegura que los valores tengan cuatro caracteres
            if len(value) == 3:
                value = '0' + value
            # Inserta ':' antes del penúltimo caracter
            value = value[:-2] + ':' + value[-2:]
        return value

    # Aplica la funcion a todas las columnas excepto la primera
    df.iloc[:, 1:] = df.iloc[:, 1:].applymap(format_time)

    # Vamos a transformar los datos en formato hora

    for col in df.columns[1:]:  # Ignora la primera columna ('Dia')
        # Convertir a Timedelta
        df[col] = pd.to_timedelta(df[col] + ':00')

    # Lista de meses abreviados
    meses = ["Ene", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
    # Crear el nuevo DataFrame con la columna 'Dia'
    nuevo_df = pd.DataFrame()
    nuevo_df['Dia'] = df['Dia']
    nuevo_df['Ciudad'] = ciudad

    # Calcular la duracion de cada mes
    for mes in meses:
        nuevo_df[mes] = (df[f'{mes}_Ocas'] - df[f'{mes}_Ort']).apply(lambda x: x.total_seconds() / 3600)

    return nuevo_df

hola = sacar_datos_horarios('Sevilla')

provincias = ['Almeria','Cadiz','Cordoba','Granada','Huelva','Jaen',
            'Malaga','Sevilla','Huesca','Zaragoza','Teruel','Oviedo',
            'Barcelona','Tarragona','Girona','Lleida','Coruna','Lugo',
            'Ourense','Pontevedra','Santander','Albacete','Ciudad',
            'Cuenca','Guadalajara','Toledo','Avila','Burgos','Leon','Salamanca',
            'Segovia','Valladolid','Zamora','Alicante','Valencia','Castellon',
            'Palma','Palmas','Santa','Logrono','Bilbao','Donostia','Vitoria',
            'Caceres','Badajoz','Madrid','Pamplona','Murcia']

lista_dataframes = []

for provincia in provincias:
    try:
        df_provincia = sacar_datos_horarios(provincia)
        lista_dataframes.append(df_provincia)
    except Exception as e:
        print(f"Datos de {provincia} no encontrados")

# Concatenar todos los DataFrames de la lista en uno solo
df_provincias = pd.concat(lista_dataframes, ignore_index=True)

# Guardar el DataFrame final en un archivo CSV
df_provincias.to_csv('datos_provincias.csv', index=False)

print("Proceso completado. El DataFrame se ha guardado en 'datos_provincias.csv'")



Proceso completado. El DataFrame se ha guardado en 'datos_provincias.csv'
